In [7]:
import os
import joblib
from time import time

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, TimeDistributed
from tensorflow.keras import Input, Model, callbacks
from tensorflow.keras.utils import plot_model as plm
from tensorflow.keras.activations import swish
from keras.callbacks import CSVLogger

from tensorflow.compat.v1.keras.backend import set_session
from scikeras.wrappers import KerasRegressor
from tcn import TCN, tcn_full_summary

import sys
sys.path.insert(0, '../MODL/INC/')
from data_load import data_load


#-------------------------------------------------------------------------
# .. Set configure

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))


#-------------------------------------------------------------------------
# .. Data set

element = 'ALLV'
name_list = "./SHEL/namelist.input"

hp_lr = 0.009
hp_pd = 'same'
hp_ns = 1
hp_dl = [1,2,4,8,16,32,48]
hp_ldl = hp_dl[-1] # last dilation factor to make name of save model
hp_bn = True
hp_nf = 80
hp_dr = 0.07
hp_ks = 6

input_size = 6
output_size = 2
num_fct = 48
batch_size = 8
n_iter_search = 20
num_epoch = 600
dev_stn_id = 47105
tran_data_per = "2101_2104_2201_2204"

exp_name = "CNTL"
csv_outdir = './DAOU/LOSS/' + exp_name + '/'
model_outdir = './DAOU/MODL/' + exp_name + '/'
scalr_outdir = './DAOU/SCAL/' + exp_name + '/'
gifd_outdir = './GIFD/' + exp_name + '/'
log_outdir = './DAOU/LOGF/' + exp_name + '/'


if os.path.exists(csv_outdir) != True: os.makedirs(csv_outdir)
if os.path.exists(model_outdir) != True: os.makedirs(model_outdir)
if os.path.exists(scalr_outdir) != True: os.makedirs(scalr_outdir)
if os.path.exists(gifd_outdir) != True: os.makedirs(gifd_outdir)
if os.path.exists(log_outdir) != True: os.makedirs(log_outdir)


nwp_file = "../DAIO/nwp_data_47105"
obs_file = "../DAIO/obs_data_47105"

sel_dm_nwp_train, sel_dm_nwp_test, dm_obs_train, dm_obs_test = data_load(nwp_file, obs_file)

================================================== load data shape
(868, 49, 20)
(868, 49, 2)
================================================== split data shape
(109, 49, 20)
(109, 49, 2)
(61, 49, 20)
(61, 49, 2)
결측 합계:  1
shape of after drop
(108, 49, 20)
(108, 49, 2)
결측 합계:  5
shape of after drop
(56, 49, 20)
(56, 49, 2)
================================================== drop data shape
(108, 48, 6)
(108, 48, 2)
(56, 48, 6)
(56, 48, 2)


In [9]:
sys.path.insert(0, '../MODL/INC/')
from data_scaling import data_scaling
from step_sampling_for_date import step_sampling_for_date
from hist_and_kde_for_split import hist_and_kde_for_split, hist_and_kde_for_split_UV

#-------------------------------------------------------------------------
# .. Model & Scaler list load

output_size = 2
tran_rate = 0.8
nbin = 10
random_seed = 1
exp_name = "test"



nor_dm_nwp_train, nor_dm_nwp_train, nor_dm_nwp_test, nor_dm_nwp_test, nor_dm_obs_train, nor_dm_obs_train,nor_dm_obs_test,nor_dm_obs_test = data_scaling(output_size, tran_rate, nbin, random_seed, exp_name, sel_dm_nwp_train, sel_dm_nwp_test,dm_obs_train, dm_obs_test)


#-------------------------------------------------------------------------
# .. split train --> train/valid for hyper-parameter tuning
#tran_x, tran_y, vald_x, vald_y = data_split_5years(re_nwp_stn, re_obs_stn, tran_rate, random_seed) 
tran_x, tran_y, vald_x, vald_y = step_sampling_for_date(nor_dm_nwp_train, nor_dm_obs_train, tran_rate, nbin, random_seed)
#-------------------------------------------------------------------------
# .. Plot tran valid
#var_name = ['NDNSW_surface', 'UGRD_10m', 'VGRD_10m', 'RH_1_5ma', 'MAXGUST_0m', 'PRMSL_meansealevel', "OBS"]
hist_and_kde_for_split_UV(exp_name, tran_data_per, dev_stn_id, gifd_outdir, tran_y, vald_y, nor_dm_obs_test)








---------- Final training data shape
<class 'numpy.ndarray'>
tran nwp :  (108, 48, 6)
tran obs :  (108, 48, 2)
test nwp :  (56, 48, 6)
test obs :  (56, 48, 2)
in split, re_nwp_stn shape:  (108, 48, 6)
tran nwp shape:  (86, 48, 6) tran obs shape:  (86, 48, 2)
vald nwp shape:  (22, 48, 6) vald obs shape:  (22, 48, 2)
plotting KDE .. 
tran_y shape:  (86, 48, 2)
vald_y shape:  (22, 48, 2)
test_y shape:  (56, 48, 2)
tran_obs shape: (86, 48, 1)
vald_obs shape: (22, 48, 1)
test_obs shape: (56, 48, 1)
calc from numpy
tran_x_median: 0.5014880952380952
vald_x_median: 0.4680059523809524
test_x_median: 0.5282738095238095
./GIFD/CNTL/KDE_ecmw_tran_vald_obs_uuu2101_2104_2201_2204_47105
tran_obs shape: (86, 48, 1)
vald_obs shape: (22, 48, 1)
test_obs shape: (56, 48, 1)
calc from numpy
tran_x_median: 0.4540816326530611
vald_x_median: 0.4447278911564625
test_x_median: 0.3801020408163265
./GIFD/CNTL/KDE_ecmw_tran_vald_obs_vvv2101_2104_2201_2204_47105


In [ ]:
#-------------------------------------------------------------------------
# .. Model load

In [ ]:
#-------------------------------------------------------------------------
# .. Read stn list

# .. all station
test_x_all = np.ndarray( shape=( len(num_his), 31, num_fct, len(model_list), input_size ), dtype=np.float32 )
test_y_all = np.ndarray( shape=( len(num_his), 31, num_fct, len(model_list), output_size ), dtype=np.float32 )
pred_test_all = np.ndarray( shape=( len(num_his), 31, num_fct, len(model_list), output_size ), dtype=np.float32 )


pred_test_all.fill(np.nan)
test_x_all.fill(np.nan)
test_y_all.fill(np.nan)


#for i in range(5):    # for models
for i in range(len(model_list)):    # for models
#for i in range(dev_stn_id.shape[0]):    # for some stations
#for i in [100, 200, 300 ]:    # for some stations

    #-------------------------------------------------------------------------
    # .. Run LSTM forecast 

    if len(model_list)!=1:
       run_stn_id = mod_stn_id[i]
    else:
       run_stn_id = mod_stn_id

    print ( "=================== Start {} station".format(run_stn_id) )

    # .. clear model
    K.clear_session()

    # .. Load model 
    model_name = load_dir + mdl_dir + model_list[i]
    try:
       print ("load_model: ", model_list[i])
       model = load_model(model_name, custom_objects={'TCN':TCN} )
    except:
       print ("Error: Could not load ", model_name)
       continue       
   

    # .. Load scaler
    print ("load_scaler:", nwp_scaler_list[i])
    print ("load_scaler:", obs_scaler_list[i])
    nwp_scaler = joblib.load(load_dir + scl_dir + nwp_scaler_list[i])
    obs_scaler = joblib.load(load_dir + scl_dir + obs_scaler_list[i])


    for k in range(len(test_name)):  # for days
    #for k in [4]:  # for days

        #-------------------------------------------------------------------------
        # .. Data load

        print(test_peri_name[k])

        test_x, test_y = test_data_load(data_dir, test_peri_name[k], element,
                                        input_size, output_size, num_his[k],
                                        num_fct, run_stn_id)

        test_x = np.swapaxes(test_x,0,1)
        #test_x_ori = copy.deepcopy(test_x)
        test_y = np.swapaxes(test_y,0,1)


        print ("======= Loaded data shape ")
        print ("test_x shape: ", test_x.shape)
        print ("test_y shape: ", test_y.shape)

 
        # .. load data  
        b, s, f = test_x.shape

        #---------------------------------------------------------------------
        # .. Model run

        # .. 2021.02.25 kmk
        for j in range(b):

            # .. check missing
            nwp_count = check_missing_existence(test_x[j,:,:], test_y[j,:,:]) 
            if nwp_count > 0:
               print ( "------- nwp missing count > 0, pass ", j, '  day' )
               continue

            # .. normalize
            nor_test_x = nwp_scaler.transform(test_x[j:j+1,:,:].reshape(1*s,f))          # scaler input dim=(N,n_feature)
            nor_test_y = obs_scaler.transform(test_y[j:j+1,:,:].reshape(1*s,output_size))
            nor_test_x = nor_test_x.reshape(1,s,f)
            nor_test_y = nor_test_y.reshape(1,s,output_size)

            nor_pred_test = model.predict(nor_test_x, batch_size=1)    
            inv_pred_test = obs_scaler.inverse_transform(nor_pred_test.reshape(1*s,output_size))
            inv_pred_test = inv_pred_test.reshape(s,output_size)

            # .. Data save
            test_x_all[k,j,:,i,:] = test_x[j,:,:]  # select feature 0
            test_y_all[k,j,:,i,:] = test_y[j,:,:]  # select feature 0
            pred_test_all[k,j,:,i,:] = inv_pred_test[:,:] # select feature 0


#pred_test_all = pred_test_all[:,:,2:-2,:]
#test_x_all = test_x_all[:,:,2:-2,:]
#test_y_all = test_y_all[:,:,2:-2,:]
print ("-------------- all list fcst complete")
print ("pred_test_all: ", pred_test_all.shape)
print ("test_x_all: ", test_x_all.shape)
print ("test_y_all: ", test_y_all.shape)

#print ( pred_test_all )

if each_stn_mod == "ON":
   np.savez( (prt_outdir + 'tcnm' + '_' + total_test_peri + '_' + find_ep + '_' + find_id), value=pred_test_all, stn_id=mod_stn_id )
   np.savez( (prt_outdir + 'g128' + '_' + total_test_peri + '_' + find_ep + '_' + find_id), value=test_x_all, stn_id=mod_stn_id )
   np.savez( (prt_outdir + 'tobs' + '_' + total_test_peri + '_' + find_ep + '_' + find_id), value=test_y_all, stn_id=mod_stn_id )
else:
   np.savez( (prt_outdir + 'tcnm' + '_' + total_test_peri + '_' + find_ep ), value=pred_test_all, stn_id=mod_stn_id )
   np.savez( (prt_outdir + 'g128' + '_' + total_test_peri + '_' + find_ep ), value=test_x_all, stn_id=mod_stn_id )
   np.savez( (prt_outdir + 'tobs' + '_' + total_test_peri + '_' + find_ep ), value=test_y_all, stn_id=mod_stn_id )